In [2]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np

train = pd.read_csv('train_V2.csv')
print(train.shape)
train.head()

(4446966, 29)


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

train = reduce_mem_usage(train)

Memory usage of dataframe is 983.90 MB
Memory usage after optimization is: 288.39 MB
Decreased by 70.7%


In [5]:
train.dropna(inplace = True)

In [12]:
train['totalDistance'] = train['rideDistance'] + train['swimDistance'] + train['walkDistance']
train['rideDistance'] /= train['totalDistance']
train['swimDistance'] /= train['totalDistance']
train['walkDistance'] /= train['totalDistance']
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,totalDistance
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00000,0,0,0,60,1241,0,0,0.00000,1306,squad-fpp,28,26,-1,0,0.000000,0,0.000000,0,0,1.000000,1,1466,0.444336,244.75
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.50000,0,0,0,57,0,0,0,0.00000,1777,squad-fpp,26,25,1484,0,0.000003,0,0.007641,0,0,0.992188,5,0,0.640137,1445.00
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00000,0,0,0,47,0,0,0,0.00000,1318,duo,50,47,1491,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.775391,161.75
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90625,0,0,0,75,0,0,0,0.00000,1436,squad-fpp,31,30,1408,0,0.000000,0,0.000000,0,0,1.000000,3,0,0.166748,202.75
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00000,0,0,0,45,0,1,1,58.53125,1424,solo-fpp,97,95,1560,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.187500,49.75


In [11]:
agg_cols = ['assists', 'damageDealt', 'DBNOs', 'headshotKills', 'killPlace', 'killPoints', 'kills',
            'longestKill', 'killStreaks', 'totalDistance']

group_agg = train[agg_cols + ['groupId']].groupby(['groupId']).mean()

for col in agg_cols:
    group_agg.rename({col: col + '_group_mean'}, axis = 1, inplace = True)

group_agg.head()

,assists_group_mean,damageDealt_group_mean,DBNOs_group_mean,headshotKills_group_mean,killPlace_group_mean,killPoints_group_mean,kills_group_mean,longestKill_group_mean,killStreaks_group_mean,totalDistance_group_mean
groupId,,,,,,,,,,
00000c08b5be36,0.000000,247.125,1.666667,0.333333,50.333333,0.0,0.666667,9.046875,0.666667,271.5000
00000d1cbbc340,0.000000,173.750,0.000000,0.000000,40.000000,0.0,1.000000,1.963867,1.000000,135.7500
000025a09dd1d7,0.000000,0.000,0.000000,0.000000,72.000000,0.0,0.000000,0.000000,0.000000,110.8125
000038ec4dff53,0.666667,263.500,1.666667,0.333333,13.000000,0.0,2.000000,53.500000,1.000000,2158.0000
00003a54230763,0.500000,50.000,0.500000,0.500000,54.500000,1013.0,0.500000,1.585938,0.500000,92.6250


In [13]:
train = pd.merge(train, group_agg, how = 'left', on = 'groupId')
train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,totalDistance,assists_group_mean,damageDealt_group_mean,DBNOs_group_mean,headshotKills_group_mean,killPlace_group_mean,killPoints_group_mean,kills_group_mean,longestKill_group_mean,killStreaks_group_mean,totalDistance_group_mean
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00000,0,0,0,60,1241,0,0,0.00000,1306,squad-fpp,28,26,-1,0,0.000000,0,0.000000,0,0,1.000000,1,1466,0.444336,244.75,0.0,102.18750,0.5,0.25,47.25,1250.5,1.0,6.914062,0.25,183.00
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.50000,0,0,0,57,0,0,0,0.00000,1777,squad-fpp,26,25,1484,0,0.000003,0,0.007641,0,0,0.992188,5,0,0.640137,1445.00,0.5,142.75000,0.5,0.00,37.00,0.0,1.0,20.921875,0.75,5092.00
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00000,0,0,0,47,0,0,0,0.00000,1318,duo,50,47,1491,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.775391,161.75,0.5,107.31250,0.5,0.00,32.50,0.0,1.0,5.421875,0.50,811.00
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90625,0,0,0,75,0,0,0,0.00000,1436,squad-fpp,31,30,1408,0,0.000000,0,0.000000,0,0,1.000000,3,0,0.166748,202.75,0.0,32.90625,0.0,0.00,75.00,0.0,0.0,0.000000,0.00,202.75
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00000,0,0,0,45,0,1,1,58.53125,1424,solo-fpp,97,95,1560,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.187500,49.75,0.0,100.00000,0.0,0.00,45.00,0.0,1.0,58.531250,1.00,49.75


In [14]:
for col in agg_cols:
    train[col] /= train[col + '_group_mean']

train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,totalDistance,assists_group_mean,damageDealt_group_mean,DBNOs_group_mean,headshotKills_group_mean,killPlace_group_mean,killPoints_group_mean,kills_group_mean,longestKill_group_mean,killStreaks_group_mean,totalDistance_group_mean
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,NaN,0,0.000000,0.0,0.0,0,1.269841,0.992403,0.0,0.0,0.0,1306,squad-fpp,28,26,-1,0,0.000000,0,0.000000,0,0,1.000000,1,1466,0.444336,1.337891,0.0,102.18750,0.5,0.25,47.25,1250.5,1.0,6.914062,0.25,183.00
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0.0,0,0.641113,0.0,NaN,0,1.540541,NaN,0.0,0.0,0.0,1777,squad-fpp,26,25,1484,0,0.000003,0,0.007641,0,0,0.992188,5,0,0.640137,0.283691,0.5,142.75000,0.5,0.00,37.00,0.0,1.0,20.921875,0.75,5092.00
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,2.0,0,0.633789,0.0,NaN,0,1.446154,NaN,0.0,0.0,0.0,1318,duo,50,47,1491,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.775391,0.199463,0.5,107.31250,0.5,0.00,32.50,0.0,1.0,5.421875,0.50,811.00
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,NaN,0,1.000000,NaN,NaN,0,1.000000,NaN,NaN,NaN,NaN,1436,squad-fpp,31,30,1408,0,0.000000,0,0.000000,0,0,1.000000,3,0,0.166748,1.000000,0.0,32.90625,0.0,0.00,75.00,0.0,0.0,0.000000,0.00,202.75
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,NaN,0,1.000000,NaN,NaN,0,1.000000,NaN,1.0,1.0,1.0,1424,solo-fpp,97,95,1560,0,0.000000,0,0.000000,0,0,1.000000,2,0,0.187500,1.000000,0.0,100.00000,0.0,0.00,45.00,0.0,1.0,58.531250,1.00,49.75


In [15]:
train.fillna(0, inplace = True)

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_val = train_test_split(train, test_size = 0.05, random_state = 191)

X_train = X_train.reset_index(drop = True)
X_val = X_val.reset_index(drop = True)

print('X_train:', X_train.shape)
print('X_val:', X_val.shape)

X_train: (4224616, 40)
X_val: (222349, 40)


In [20]:
Y_train = X_train['winPlacePerc']
X_train.drop('winPlacePerc', axis=1, inplace=True)

Y_val = X_val['winPlacePerc']
X_val.drop('winPlacePerc', axis=1, inplace=True)

In [21]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

d_train = lgb.Dataset(X_train, Y_train)
d_valid = lgb.Dataset(X_val, Y_val)

watchlist = [d_train, d_valid]

print(X_train.shape)
print(Y_train.shape)

(4224616, 39)
(4224616,)


In [22]:
import gc

params = {
    'learning_rate': 0.08, 
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'num_leaves': 31,
    'lambda_l2': 0.01,
    'objective': 'regression_l2', 
    'metric': 'mae',
    'seed': 191
}

gc.collect()

260